In [1]:
import gc
import torch
import numpy as np
import pandas as pd
import functools
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Наши новые модули
import src.config as config
from src.train.ptuning_train import train
from src.prompt import mbpp, humaneval
from src.data.loader import load_benchmark
from src.executor import LocalExecutor
from src.metrics import GreedyPass, PassAtk, PercentPassed, MeanEntropy, ExecutionResult

Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


In [2]:
TRAIN = False

In [3]:
if TRAIN:
    print("🔥 Режим обучения: Запускаем train()...")
    model, tokenizer = train()

else:
    print(f"❄️ Режим инференса: Загружаем базу + P-Tuning адаптер...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        config.MODEL_PATH,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="sdpa"
    )

    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_PATH, trust_remote_code=True)

    print(f"🔗 Подключаем адаптер из: {config.PTUNING_MODEL_PATH}")
    model = PeftModel.from_pretrained(base_model, config.PTUNING_MODEL_PATH)


model.eval()
print("✅ Модель готова к тестам!")

❄️ Режим инференса: Загружаем базу + P-Tuning адаптер...
🔗 Подключаем адаптер из: ./models/qwen3-0.6b-ptuning
✅ Модель готова к тестам!


In [4]:
# Вспомогательная функция для энтропии (нужна для evaluate_transformers)
def calculate_entropy_hf(scores):
    """Считает энтропию по логитам HuggingFace (Tuple of tensors)"""
    if not scores: return 0.0

    entropies = []

    for step_logits in scores:
        # step_logits: [batch_size, vocab_size]
        # Переводим в вероятности
        probs = torch.softmax(step_logits, dim=-1)
        log_probs = torch.log_softmax(step_logits, dim=-1)

        # H = - sum(p * log p) по последней размерности (vocab)
        # Получаем [batch_size] энтропий для этого шага
        step_entropy = -(probs * log_probs).sum(dim=-1)

        entropies.append(step_entropy) # List of tensors

    # Собираем в [steps, batch_size] -> [batch_size, steps]
    if not entropies: return [0.0]

    # Stack -> [seq_len, batch_size]
    entropies_tensor = torch.stack(entropies)

    # Среднее по длине последовательности -> [batch_size]
    mean_entropies = entropies_tensor.mean(dim=0)

    return mean_entropies.tolist()

def evaluate_transformers(model, tokenizer, tasks, metrics_suite, batch_size=4): # <--- СНИЗИЛ БАТЧ ДО 4
    executor = LocalExecutor()
    final_results = {}

    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Чистим конфиг
    base_config = config.SAMPLING_SETTINGS.copy()
    for key in ["n", "temperature", "ignore_eos", "detokenize", "logprobs", "stop"]:
        base_config.pop(key, None)
    if "max_tokens" in base_config:
        base_config["max_new_tokens"] = base_config.pop("max_tokens")

    grouped = {}
    for m in metrics_suite:
        cfg_key = str(m.gen_config)
        if cfg_key not in grouped: grouped[cfg_key] = []
        grouped[cfg_key].append(m)

    for cfg_key, metrics in grouped.items():
        metric_specific_config = metrics[0].gen_config
        generation_kwargs = {**base_config, **metric_specific_config}

        if not generation_kwargs.get("do_sample", False):
            for k in ["temperature", "top_p", "top_k"]:
                generation_kwargs.pop(k, None)

        print(f"\n🚀 Generation for: {[m.name for m in metrics]} | Batch Size: {batch_size}")

        all_exec_results = []

        # tqdm с очисткой
        pbar = tqdm(range(0, len(tasks), batch_size), desc="Eval Batches")

        for i in pbar:
            try:
                batch_tasks = tasks[i : i + batch_size]
                batch_prompts = [t.prompt for t in batch_tasks]

                inputs = tokenizer(
                    batch_prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True
                ).to(model.device)

                input_len = inputs.input_ids.shape[1]

                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        pad_token_id=tokenizer.eos_token_id,
                        return_dict_in_generate=True,
                        output_scores=True, # Нужно для энтропии, но ест память
                        **generation_kwargs
                    )

                # Считаем энтропию СРАЗУ, пока тензоры в памяти, чтобы потом удалить
                batch_entropies = [0.0] * len(batch_tasks)
                if outputs.scores:
                    # scores - это tuple(len=seq_len) of tensors [batch, vocab]
                    # Наша функция вернет список [ent_sample_0, ent_sample_1, ...]
                    batch_entropies = calculate_entropy_hf(outputs.scores)

                # Важно: отвязываем энтропию от графа (хотя мы в no_grad, но на всякий случай переводим в float)
                # calculate_entropy_hf уже возвращает list[float]

                n_samples = generation_kwargs.get("num_return_sequences", 1)

                for j, task in enumerate(batch_tasks):
                    task_samples = []
                    start_idx = j * n_samples
                    end_idx = start_idx + n_samples

                    # Энтропия для текущей задачи (для greedy n=1, энтропия одна)
                    # Если n > 1, HF повторяет промпты, но outputs.scores сложнее маппятся.
                    # Для простоты считаем, что output_scores возвращает [batch_size * n_samples, vocab]
                    # calculate_entropy_hf вернет список длиной batch_size * n_samples

                    for k in range(start_idx, end_idx):
                        seq = outputs.sequences[k]
                        gen_tokens = seq[input_len:]
                        gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True)

                        # Берем энтропию для конкретного сэмпла
                        sample_entropy = batch_entropies[k] if k < len(batch_entropies) else 0.0

                        exec_res = executor.execute(gen_text, task.tests)
                        exec_res.entropy = sample_entropy
                        task_samples.append(exec_res)

                    all_exec_results.append(task_samples)

                # === АГРЕССИВНАЯ ОЧИСТКА ===
                del inputs
                del outputs # Удаляем 5 ГБ логитов
                del batch_entropies
                gc.collect()
                torch.cuda.empty_cache()
                # ===========================

            except Exception as e:
                print(f"Error in batch {i}: {e}")
                torch.cuda.empty_cache()
                continue

        for m in metrics:
            score = m.calculate(all_exec_results)
            final_results[m.name] = score
            print(f"📊 {m.name}: {score:.4f}")

    return final_results

In [5]:
# 1. Загрузка данных (через новые лоадеры)
print("📥 Loading Datasets...")

# Функция-маппер с "запеченным" токенизатором
mbpp_mapper = functools.partial(mbpp.mbpp_to_task, tokenizer=tokenizer)
he_mapper = functools.partial(humaneval.humaneval_to_task, tokenizer=tokenizer)

# Грузим сырые данные
ds_mbpp = mbpp.get_dataset()["test"]
ds_mbpp_small = ds_mbpp#.select(range(20))

ds_he = humaneval.get_dataset()["test"]
ds_he_small = ds_he#.select(range(20))

tasks_mbpp = load_benchmark(ds_mbpp_small, mbpp_mapper)
tasks_he = load_benchmark(ds_he_small, he_mapper)

metrics_suite = [
	GreedyPass(),
    PassAtk(k=1, n_samples=1),
    # PassAtk(k=5, n_samples=5), # оно просто не запускается, память умирает
	PercentPassed(),
    MeanEntropy(),
]

# 3. Запуск!
print("\n" + "="*40)
print("🐍 EVALUATING MBPP (Small Subset)")
print("="*40)
results_mbpp = evaluate_transformers(model, tokenizer, tasks_mbpp, metrics_suite)

print("\n" + "="*40)
print("🧠 EVALUATING HumanEval (Small Subset)")
print("="*40)
results_he = evaluate_transformers(model, tokenizer, tasks_he, metrics_suite)

# 4. Итоговый вывод
print("\n=== FINAL REPORT ===")
df = pd.DataFrame([results_mbpp, results_he], index=["MBPP", "HumanEval"])
display(df)

📥 Loading Datasets...

🐍 EVALUATING MBPP (Small Subset)

🚀 Generation for: ['greedy@1'] | Batch Size: 4


Eval Batches:   0%|          | 0/65 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/opt/conda/lib/python3.11/site-packages/peft/peft_model.py:2141: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


📊 greedy@1: 0.2568

🚀 Generation for: ['pass@1 (n=1)', 'mean_%passed', 'mean_entropy'] | Batch Size: 4


Eval Batches:   0%|          | 0/65 [00:00<?, ?it/s]

...

Eval Batches:   0%|          | 0/41 [00:00<?, ?it/s]

7.5
7.5
7.5
📊 greedy@1: 0.2195

🚀 Generation for: ['pass@1 (n=1)', 'mean_%passed', 'mean_entropy'] | Batch Size: 4


Eval Batches:   0%|          | 0/41 [00:00<?, ?it/s]

2
3
2
3
2
3
2
3
2
3
2
3
2
3
2
3
📊 pass@1 (n=1): 0.2317
📊 mean_%passed: 0.3919
📊 mean_entropy: nan

=== FINAL REPORT ===


,greedy@1,pass@1 (n=1),mean_%passed,mean_entropy
MBPP,0.256809,0.249027,0.297990,NaN
HumanEval,0.219512,0.231707,0.391887,NaN
